### Importing necessary libraries 

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
from scipy import stats

import os
import warnings
warnings.filterwarnings('ignore')

### PARAMETERS 

In [2]:
path = "C:/Users/prash/Downloads/STOCK MARKET/"

In [3]:
stock = 'ASIANPAINT'

### Reading the historical share prices (CSV)

In [4]:
share_price = pd.read_csv(path + "MONTHLY SHARE PRICES/" + stock + ".csv")

share_price['Year'] = pd.to_datetime(share_price['Date']).dt.to_period('Y')

### Crawling the SCREENER website 

In [5]:
url = ('https://www.screener.in/company/' + stock)

page = urlopen(url)

soup = BeautifulSoup(page, 'html.parser')

### Extracting the Book Value from SCREENER

In [6]:
summary_section = soup.findAll('li', {"class" : 'four columns'})

for li in summary_section:
    li_name = li.find('b')
    if( (np.array(li.text.split())[0]) == "Book"):
        book_value = float(li_name.text)
book_value

98.23

### Extracting the Market Value from SCREENER

In [7]:
summary_section = soup.findAll('li', {"class" : 'four columns'})

for li in summary_section:
    li_name = li.find('b')
    if( (np.array(li.text.split())[0]) == "Market"):
        market_value = (li_name.text)
        market_value = float(market_value.replace(',',''))
market_value

172584.0

### Extracting the CMP(Current Market Price) from SCREENER

In [8]:
summary_section = soup.findAll('li', {"class" : 'four columns'})

for li in summary_section:
    li_name = li.find('b')
    if( (np.array(li.text.split())[0]) == "Current"):
        cmp = (li_name.text)
        cmp = float(cmp.replace(',',''))
cmp

1799.0

### Extracting the Profit-Loss section/table from SCREENER

In [9]:
profit_loss_section = soup.findAll('section', {"id" : 'profit-loss'})

profit_loss_df = []

for section in profit_loss_section:
    col_names = section.find('thead')
    col_names = col_names.find_all('th')
    df = [cell.text.strip() for cell in col_names]
    profit_loss_df.append(df)
    
for section in profit_loss_section:
    table_body = section.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        row_vals = row.find_all('td')
        df1 = [cell.text.strip() for cell in row_vals] 
        profit_loss_df.append(df1)
        
profit_loss_df = pd.DataFrame(profit_loss_df)

profit_loss_df.columns = profit_loss_df.iloc[0]

profit_loss_df = profit_loss_df.drop(profit_loss_df.index[0])

### Extracting the Balance Sheet section/table from SCREENER

In [10]:
balance_sheet_section = soup.findAll('section', {"id" : 'balance-sheet'})

balance_sheet_df = []

for section in balance_sheet_section:
    col_names = section.find('thead')
    col_names = col_names.find_all('th')
    df = [cell.text.strip() for cell in col_names]
    balance_sheet_df.append(df)
    
for section in balance_sheet_section:
    table_body = section.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        row_vals = row.find_all('td')
        df1 = [cell.text.strip() for cell in row_vals] 
        balance_sheet_df.append(df1)
        
balance_sheet_df = pd.DataFrame(balance_sheet_df)

balance_sheet_df.columns = balance_sheet_df.iloc[0]

balance_sheet_df = balance_sheet_df.drop(balance_sheet_df.index[0])

balance_sheet_df.drop(balance_sheet_df.columns[len(balance_sheet_df.columns)-1], axis=1, inplace=True)

### Total Liabilities from Balance Sheet Dataframe 

In [11]:
total_liabilities = balance_sheet_df.iloc[4][len(balance_sheet_df.columns)-1]

total_liabilities = float(total_liabilities.replace(',',''))

### Total Sales from Profit-Loss Dataframe 

In [12]:
total_sales = profit_loss_df.iloc[0][len(profit_loss_df.columns)-2]

total_sales = float(total_sales.replace(',',''))

### Yearly EPS(Earnings Per Share) Data from Profit-Loss Dataframe 

In [13]:
years_column = pd.DataFrame(profit_loss_df.columns[:-1])
years_column.columns = ["Name"]
years_column[["Month","Year"]] = years_column.Name.str.split(expand=True) 


yearly_eps = pd.DataFrame(profit_loss_df.iloc[10])
yearly_eps = yearly_eps[:-1]

yearly_eps["Year"] = np.array(years_column["Year"])

yearly_eps = yearly_eps.drop(yearly_eps.index[0])

yearly_eps = pd.DataFrame(np.array(yearly_eps))

yearly_eps.columns = ["EPS","Year"]

yearly_eps["Year"] = yearly_eps["Year"].astype(str).astype(float)


### Yearly CMP (Current Market Price) from Share Price Dataframe 

In [14]:
yearly_cmp = share_price.groupby('Year', as_index=False)['Close'].mean()

yearly_cmp["Year"] = yearly_cmp["Year"].astype(str).astype(int)

### Calculating Yearly P/E Ratio and EPS-GR (EPS Growth Rate)

In [15]:
yearly_p_by_e_ratio = pd.merge(yearly_cmp,yearly_eps,on='Year', how='left')

yearly_p_by_e_ratio = yearly_p_by_e_ratio.dropna()

yearly_p_by_e_ratio["EPS"] = yearly_p_by_e_ratio["EPS"].astype(str).astype(float)

yearly_p_by_e_ratio["P/E"] = yearly_p_by_e_ratio["Close"]/yearly_p_by_e_ratio["EPS"]

yearly_p_by_e_ratio["EPS GR"] = 0

for i in range(1,yearly_p_by_e_ratio.shape[0]):
    yearly_p_by_e_ratio["EPS GR"].iloc[i] = ((yearly_p_by_e_ratio["EPS"].iloc[i] - yearly_p_by_e_ratio["EPS"].iloc[i-1])/yearly_p_by_e_ratio["EPS"].iloc[i-1])*100

In [16]:
#yearly_p_by_e_ratio

### Removing Outliers 

In [17]:
yearly_p_by_e_ratio = yearly_p_by_e_ratio[(np.abs(stats.zscore(yearly_p_by_e_ratio)) < yearly_p_by_e_ratio.shape[1]).all(axis=1)]

### Finding the P/E GR 

In [18]:
p_by_e_gr = (yearly_p_by_e_ratio['P/E'].iloc[yearly_p_by_e_ratio.shape[0]-1] / yearly_p_by_e_ratio['P/E'].iloc[0]) ** (1/yearly_p_by_e_ratio.shape[0])
p_by_e_gr = p_by_e_gr - 1

In [19]:
eps_gr = (yearly_p_by_e_ratio['EPS'].iloc[yearly_p_by_e_ratio.shape[0]-1] / yearly_p_by_e_ratio['EPS'].iloc[0]) ** (1/yearly_p_by_e_ratio.shape[0])
eps_gr = eps_gr - 1

In [20]:
eps_gr

0.11913516374810418

### Finding the Avg P/E (OR) Intrinsic P/E

In [21]:
avg_p_by_e = np.sum(yearly_p_by_e_ratio['P/E'])/yearly_p_by_e_ratio.shape[0]

### Finding the Best Case P/E 

In [22]:
best_case_p_by_e = yearly_p_by_e_ratio['P/E'].iloc[yearly_p_by_e_ratio.shape[0]-1]*(1 + p_by_e_gr)

### Finding the Intrinsic Value 

In [23]:
intrinsic_value = avg_p_by_e*float(yearly_eps['EPS'][yearly_eps.shape[0]-1])
intrinsic_value

1414.2607951485438

### Finding the Most Optimistic Value

In [24]:
most_optimistic_value = best_case_p_by_e*float(yearly_eps['EPS'][yearly_eps.shape[0]-1])
most_optimistic_value

1531.5056823963937

In [25]:
cmp > intrinsic_value

True

### PEG Ratio - Need below 2 or as low as possible 

In [26]:
peg_ratio = avg_p_by_e / (eps_gr*100)
peg_ratio

5.33291149686625

In [27]:
peg_ratio>2

True

### P/B Ratio (Price to Book Value) - Need below 1 or as low as possible

In [28]:
p_by_b_ratio = cmp / book_value
p_by_b_ratio

18.314160643387968

### ROE(Return On Equity) - Need above 15% or as high as possible

In [29]:
roe = (float(yearly_eps['EPS'][yearly_eps.shape[0]-1]) / book_value)*100
roe

22.661101496487834

### D/E Ratio (Debt to Equity) - Need as low as possible (Industry Specific)

In [30]:
d_by_e_ratio = total_liabilities/(book_value * (market_value/cmp) )
d_by_e_ratio

1.4012799059932444

### PSR (Price to Sales Ratio) - Need below 2 or as low as possible 

In [31]:
psr = market_value/total_sales
psr

10.528550512445095